# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['AngleX', 'AngleY', 'LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [40]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [41]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Stand', 'Left', 'Crouch', 'Sit', 'Belly', 'Knee', 'StandInit', 'Back', 'Right', 'Frog', 'HeadBack']


In [42]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [43]:
# load all the data
all_data = []
all_target = []

for i in range(0,len(classes)):
    data,target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    
    
print 'total number of data', len(all_data)

total number of data 200


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [44]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.5)
training_data = permutation[:n_training_data]

In [45]:
print training_data

[145 148 176 134 110 165 174   7  99  48  74  93  86  15  72 153  68   0
 128 158 107  55  41 196 121  70  98  10 150 163 161  96  27 177  47 151
 156   3  85  38 171 167  81 125  32   9  89 170 187   1 114  76 159 129
  20 183 155 175  87 178  83  62  59  73 136  50   6  58 141  80  95 106
  63 172  94 169  88  26  46  84  67 140  97 126 190  25  56 108 166  75
 104  43  90 132  23 194 147 124 191  22]


In [46]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [47]:
fit_data = []
fit_target = []
for i in range(0,len(training_data)):
    fit_data.append(all_data[training_data[i]])
    fit_target.append(all_target[training_data[i]])
    
clf.fit(fit_data, fit_target) 

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

### predicting

In [48]:
clf.predict(all_data[1]), all_target[1]

(array([0]), 0)

In [49]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [50]:
expected = []
predicted = []
prediction_data = []
for i in range(0,len(all_data)):
    if( not (i in training_data)):
        expected.append(all_target[i])
        prediction_data.append(all_data[i])
predicted = clf.predict(prediction_data)

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         4
          1       1.00      0.38      0.56        13
          2       0.51      1.00      0.68        18
          3       1.00      1.00      1.00        10
          4       1.00      1.00      1.00         2
          5       1.00      1.00      1.00         5
          6       1.00      1.00      1.00        28
          7       1.00      1.00      1.00         4
          8       1.00      0.50      0.67         2
          9       0.00      0.00      0.00         8
         10       1.00      1.00      1.00         6

avg / total       0.83      0.83      0.80       100


Confusion matrix:
[[ 4  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  8  0  0  0  0  0  0  0  0]
 [ 0  0 18  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0  0 28  0  0  0  0]
 [ 0  0  0  0 

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [51]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [52]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[1]), all_target[1]

(array([0]), 0)